In [32]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [33]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000578616s, CPU 0.00058s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0144194s, CPU 0.012958s)
Info    : 98 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [34]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:PlaneStress)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 16)


Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 98)

In [35]:
supp = FEM.displacementConstraint("left", ux=0, uy=0)
load = FEM.load("right", fx=1)

("right", 1, 0, 0)

In [36]:
u0 = FEM.initialDisplacement(problem, "body", ux=0, uy=0)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :u2D, Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 98))

In [37]:
FEM.initialDisplacement!("left", u0, ux=0, uy=0)
u0

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :u2D, Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 98))

In [38]:
v0 = FEM.initialVelocity(problem, "body", vx=0, vy=0)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :v2D, Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 98))

In [39]:
FEM.initialVelocity!("body", v0, vx=0, vy=0)

In [40]:
K = FEM.stiffnessMatrix(problem)
M = FEM.massMatrix(problem)
f = FEM.loadVector(problem, [load])

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :f2D, Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 98))

In [41]:
FEM.applyBoundaryConditions!(K, M, f, [supp])

In [42]:
Tmin = FEM.smallestPeriodTime(K, M)

4.735883743230799e-8

In [43]:
u, v = FEM.CDM(K, M, f, u0, v0, 10Tmin, Tmin / π)

(LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 6.914952908823762e-6 6.5918733074658006e-6; 0.0 0.0 … -8.090808616938553e-7 -8.466065569159533e-7], [0.0, 1.5074786152874603e-8, 3.0149572305749206e-8, 4.522435845862381e-8, 6.029914461149841e-8, 7.537393076437302e-8, 9.044871691724763e-8, 1.0552350307012224e-7, 1.2059828922299685e-7, 1.3567307537587146e-7  …  3.3164529536324136e-7, 3.4672008151611597e-7, 3.617948676689906e-7, 3.768696538218652e-7, 3.919444399747398e-7, 4.070192261276144e-7, 4.22094012280489e-7, 4.371687984333636e-7, 4.5224358458623824e-7, 4.6731837073911284e-7], Int64[], 32, :u2D, Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 98)), LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … -14.255649512817172 -21.43178669

In [44]:
FEM.showDoFResults(u, :uvec, visible=true)

0

In [45]:
FEM.showDoFResults(v, :vvec)

1

In [46]:
e1 = FEM.solveStrain(u)

LowLevelFEM.TensorField([[0.0 0.0 … 5.052949475771428e-6 4.351373994791214e-6; 0.0 0.0 … 2.8920792316562026e-7 2.408865222469948e-7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -1.2400769957860867e-6 -8.90033050364571e-7], [0.0 0.0 … 5.009619114111695e-6 4.710365174033526e-6; 0.0 0.0 … 4.239050681162101e-7 2.1655632185136798e-7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -1.0887365754850831e-6 -7.187288823801524e-7], [0.0 0.0 … 4.946317515040217e-6 5.239553948915637e-6; 0.0 0.0 … -1.8792182733832022e-7 -1.0576431618140105e-7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -1.890255920676126e-6 -1.5903641971704737e-6], [0.0 0.0 … 9.941522495668136e-6 1.0865528769182448e-5; 0.0 0.0 … -2.2957917960472653e-7 -2.954608128370044e-7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -3.708988253292701e-6 -4.0221428624980146e-6], [0.0 0.0 … 5.097605215650594e-6 5.55913845859102e-6; 0.0 0.0 … -3.7049488194303725e-7 -7.818396733481833e-8; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -1.511024855038121e-6 -1.3826127750192755e-6], [0.0 0.0 … 9.262941

In [47]:
FEM.showStrainResults(e1, :e)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


2

In [48]:
e2 = FEM.solveStrain(u, DoFResults=true)

LowLevelFEM.TensorField(Matrix{Float64}[], [0.0 0.0 … 1.0681415652682002e-5 1.1208790487106547e-5; 0.0 0.0 … 5.33976647397941e-6 5.180970383744655e-6; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … -1.6457531929030509e-6 -1.5710468684099549e-6], [0.0, 1.5074786152874603e-8, 3.0149572305749206e-8, 4.522435845862381e-8, 6.029914461149841e-8, 7.537393076437302e-8, 9.044871691724763e-8, 1.0552350307012224e-7, 1.2059828922299685e-7, 1.3567307537587146e-7  …  3.3164529536324136e-7, 3.4672008151611597e-7, 3.617948676689906e-7, 3.768696538218652e-7, 3.919444399747398e-7, 4.070192261276144e-7, 4.22094012280489e-7, 4.371687984333636e-7, 4.5224358458623824e-7, 4.6731837073911284e-7], Int64[], 32, :e, Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 98))

In [49]:
FEM.showDoFResults(e2, :e)

3

In [50]:
e3 = FEM.elementsToNodes(e1)

LowLevelFEM.TensorField(Matrix{Float64}[], [0.0 0.0 … 1.0681415652682002e-5 1.1208790487106547e-5; 0.0 0.0 … 2.669883236989705e-6 2.5904851918723277e-6; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … -1.6457531929030509e-6 -1.5710468684099549e-6], [0.0, 1.5074786152874603e-8, 3.0149572305749206e-8, 4.522435845862381e-8, 6.029914461149841e-8, 7.537393076437302e-8, 9.044871691724763e-8, 1.0552350307012224e-7, 1.2059828922299685e-7, 1.3567307537587146e-7  …  3.3164529536324136e-7, 3.4672008151611597e-7, 3.617948676689906e-7, 3.768696538218652e-7, 3.919444399747398e-7, 4.070192261276144e-7, 4.22094012280489e-7, 4.371687984333636e-7, 4.5224358458623824e-7, 4.6731837073911284e-7], Int64[], 32, :e, Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 98))

In [51]:
FEM.showDoFResults(e3, :e)

4

In [52]:
s1 = FEM.solveStress(u)

LowLevelFEM.TensorField([[0.0 0.0 … 0.968157918701156 0.8063703146393516; 0.0 0.0 … 0.04449352664086466 0.03705946496107613; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.9382085676302734 0.8352452394482583; 0.0 0.0 … 0.06521616432557079 0.03331635720790276; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 1.051780528571745 1.0507566378594018; 0.0 0.0 … -0.02891105035974157 -0.016271433258677085; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 2.1000785767632055 2.290411020258533; 0.0 0.0 … -0.03531987378534254 -0.04545550966723145; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 1.0167123185674944 1.0679617282477378; 0.0 0.0 … -0.056999212606621115 -0.012028302666895127; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 1.9818042101630922 2.072998762943155; 0.0 0.0 … 9.932078068419725e-5 -0.048799758870449045; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.3653429678701636 … 0.6935312549187367 0.883088783058594; 0.0 -0.009281002657059015 … -0.03351397

In [53]:
FEM.showStressResults(s1, :s)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


5

In [54]:
s2 = FEM.solveStress(u, DoFResults=true)

LowLevelFEM.TensorField(Matrix{Float64}[], [0.0 0.0 … 2.2727796225806403 2.382421304852839; 0.0 0.0 … 0.4107512672291853 0.3985361833649735; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0, 1.5074786152874603e-8, 3.0149572305749206e-8, 4.522435845862381e-8, 6.029914461149841e-8, 7.537393076437302e-8, 9.044871691724763e-8, 1.0552350307012224e-7, 1.2059828922299685e-7, 1.3567307537587146e-7  …  3.3164529536324136e-7, 3.4672008151611597e-7, 3.617948676689906e-7, 3.768696538218652e-7, 3.919444399747398e-7, 4.070192261276144e-7, 4.22094012280489e-7, 4.371687984333636e-7, 4.5224358458623824e-7, 4.6731837073911284e-7], Int64[], 32, :s, Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 98))

In [55]:
FEM.showDoFResults(s2, :s)

6

In [56]:
err1 = FEM.fieldError(e1)

LowLevelFEM.TensorField(Matrix{Float64}[], [0.0 0.0 … 7.520007163605161e-7 9.661164313631103e-7; 0.0 0.0 … 1.9111359171989142e-7 1.315643676953329e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 4.1546572197856127e-7 2.66887607807384e-7], [0.0, 1.5074786152874603e-8, 3.0149572305749206e-8, 4.522435845862381e-8, 6.029914461149841e-8, 7.537393076437302e-8, 9.044871691724763e-8, 1.0552350307012224e-7, 1.2059828922299685e-7, 1.3567307537587146e-7  …  3.3164529536324136e-7, 3.4672008151611597e-7, 3.617948676689906e-7, 3.768696538218652e-7, 3.919444399747398e-7, 4.070192261276144e-7, 4.22094012280489e-7, 4.371687984333636e-7, 4.5224358458623824e-7, 4.6731837073911284e-7], Int64[], 32, :e, Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 98))

In [57]:
FEM.showDoFResults(err1, :tensor)

7

In [58]:
u2, v2 = FEM.HHT(K, M, f, u0, v0, 10Tmin, Tmin / π)

(LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 1.3933835462338521e-9 … 6.905143783536989e-6 6.663672475393901e-6; 0.0 -1.3326588533142161e-10 … -7.659203887005172e-7 -8.245059023733488e-7], [0.0, 1.5074786152874603e-8, 3.0149572305749206e-8, 4.522435845862381e-8, 6.029914461149841e-8, 7.537393076437302e-8, 9.044871691724763e-8, 1.0552350307012224e-7, 1.2059828922299685e-7, 1.3567307537587146e-7  …  3.3164529536324136e-7, 3.4672008151611597e-7, 3.617948676689906e-7, 3.768696538218652e-7, 3.919444399747398e-7, 4.070192261276144e-7, 4.22094012280489e-7, 4.371687984333636e-7, 4.5224358458623824e-7, 4.6731837073911284e-7], Int64[], 32, :u2D, Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 98)), LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0

In [59]:
FEM.showDoFResults(u2, :uvec)

8

In [60]:
FEM.showDoFResults(v2, :vvec)

9

In [61]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [62]:
gmsh.finalize()